In [ ]:
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR

In [ ]:
batch_size = 32
num_classes = 10
learning_rate = 0.001
step_size = 16
gamma = 0.7
weight_decay = 0.00001
num_epochs = 70
model_name = 'pytorch_cifar10_model.pth'
device = 'cuda:0'

In [ ]:
# The data, shuffled and split between train and test sets:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])
train_set = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# one-hot encoding
def to_categorical_all(categories):
    num_items = len(categories)
    binary_categories = torch.zeros((num_items,num_classes),dtype=torch.float)
    for i in range(num_items):
        binary_categories[i,categories[i].item()] = 1
    return binary_categories

In [ ]:
# network definition
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3,32,(3,3),padding=1), # B x 3 x 32 x 32 -> B x 32 x 32 x 32
            nn.ReLU(),
            nn.Conv2d(32,32,(3,3)),          # B x 32 x 32 x 32 -> B x 32 x 30 x 30
            nn.ReLU(),
            nn.MaxPool2d(2),                 # B x 32 x 30 x 30 -> B x 32 x 15 x 15
            nn.Dropout(0.25),
            #----------------
            nn.Conv2d(32,64,(3,3),padding=1),# B x 32 x 15 x 15 -> B x 64 x 15 x 15
            nn.ReLU(),
            nn.Conv2d(64,64,(3,3)),          # B x 64 x 15 x 15 -> B x 64 x 13 x 13
            nn.ReLU(),
            nn.MaxPool2d(2),                 # B x 64 x 13 x 13 -> B x 64 x 6 x 6
            nn.Dropout(0.25),
            #----------------
            nn.Conv2d(64,64,(3,3),padding=1),# B x 64 x 6 x 6 -> B x 64 x 6 x 6
            nn.ReLU(),
            nn.Conv2d(64,64,(3,3)),          # B x 64 x 6 x 6 -> B x 64 x 4 x 4
            nn.ReLU(),
            nn.MaxPool2d(2),                 # B x 64 x 4 x 4 -> B x 64 x 2 x 2
            nn.Dropout(0.25),
            #----------------
            nn.Flatten()                     # B x 64 x 2 x 2 -> 256
        )
        self.perceptron = nn.Sequential(
            nn.Linear(256,512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512,num_classes),
            #nn.Softmax(1)
        )
    def forward(self, x):
        features = self.encoder(x)
        return self.perceptron(features)

In [ ]:
model = Classifier().to(device)

In [ ]:
# Apply Xavier initialization
def initialize_weights(m):
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
#
model.apply(initialize_weights)

In [ ]:
# Define loss funcion
criterion = nn.CrossEntropyLoss()

In [ ]:
# Define optimizer
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [ ]:
scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

In [ ]:
# Training
history_loss = []
history_acc = []
history_test_acc = []
epoch = 0

In [ ]:
def train(num_epochs):
    global epoch
    for _ in range(num_epochs):
        # change model in training mood
        model.train()

        # to record loss and accuracy
        batch_loss = []
        total_train = 0
        correct_train = 0

        for batch, (x_train, y_train) in enumerate(train_loader):

            # send data to device
            input = x_train.to(device)

            # reset parameters gradient to zero
            optimizer.zero_grad()

            # forward pass to the model
            output = model(input)

            # categorization
            expected_output = y_train.to(device)

            # cross entropy loss
            loss = criterion(output, expected_output)

            # find gradients
            loss.backward()
            # update parameters using gradients
            optimizer.step()

            # recording loss
            batch_loss.append(loss.item())

            # recording accuracy
            total_train += output.shape[0]
            correct_train += torch.argmax(output,dim=1).to('cpu').eq(y_train).sum().item()

        epoch_loss = np.average(batch_loss)
        epoch_acc = (100.0 * correct_train) / total_train

        history_loss.append(epoch_loss)
        history_acc.append(epoch_acc)

        total_test = 0
        correct_test = 0

        model.eval()

        for batch, (x_test, y_test) in enumerate(test_loader):

            # send data to device
            input = x_test.to(device)

            # forward pass to the model
            with torch.no_grad():
                output = model(input)

            # recording accuracy
            total_test += output.shape[0]
            correct_test += torch.argmax(output,dim=1).to('cpu').eq(y_test).sum().item()

        test_acc = (100.0 * correct_test) / total_test

        history_test_acc.append(test_acc)

        print(f'Epoch: {epoch} Loss: {epoch_loss:.6f} Accuracy: {epoch_acc:.4f} Test accuracy: {test_acc:.4f} Learning Rate: {scheduler.get_last_lr()[0]:.7f}')
        scheduler.step()
        epoch += 1

In [ ]:
print('\nTraining')
train(num_epochs)

In [ ]:
# Save model and weights
def save():
    torch.save(model.state_dict(), model_name) # weights only
    print('Saved trained model at %s ' % model_name)

In [ ]:
save()

In [ ]:
def validate():
    model.eval()
    total_train = 0
    correct_train = 0
    for batch, (x_train, y_train) in enumerate(train_loader):
        input = x_train.to(device)
        with torch.no_grad():
            output = model(input)
        total_train += output.shape[0]
        correct_train += torch.argmax(output,dim=1).to('cpu').eq(y_train).sum().item()
    acc_train = (100.0 * correct_train) / total_train
    print(f'training accuracy: {acc_train:.2f}%')
    total_test = 0
    correct_test = 0
    for batch, (x_test, y_test) in enumerate(test_loader):
        input = x_test.to(device)
        with torch.no_grad():
            output = model(input)
        total_test += output.shape[0]
        correct_test += torch.argmax(output,dim=1).to('cpu').eq(y_test).sum().item()
    acc_test = (100.0 * correct_test) / total_test
    print(f'testing accuracy: {acc_test:.2f}%')

In [ ]:
validate()

In [ ]:
# test
def test():
    dataiter = iter(train_loader)
    sample_inputs, sample_outputs = next(dataiter)
    model.eval() # switch off dropout
    with torch.no_grad():
        logits = model(sample_inputs.to(device))
        categories = torch.argmax(logits,dim=1)
        probabilities = F.softmax(logits,dim=1)
    print(sample_outputs)
    print(categories)
    print(sample_outputs[17],'...',[f"{probability:.3f}" for probability in probabilities[17]])
    print(sample_outputs[17],'...',[f"{probability:.3f}" for probability in to_categorical_all(sample_outputs)[17]])

In [ ]:
test()

In [ ]:
def history():
    # optional only:
    import matplotlib.pyplot as plt
    # Loss Curves
    plt.figure(figsize=[8,6])
    plt.plot(history_loss,'r',linewidth=3.0)
    plt.legend(['Training loss'],fontsize=18)
    plt.xlabel('Epochs ',fontsize=16)
    plt.ylabel('Loss',fontsize=16)
    plt.title('Loss Curves',fontsize=16)
    plt.savefig('fig1.png')
    # Accuracy Curves
    plt.figure(figsize=[8,6])
    plt.plot(history_acc,'r',linewidth=3.0)
    plt.plot(history_test_acc,'b',linewidth=3.0)
    plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
    plt.xlabel('Epochs ',fontsize=16)
    plt.ylabel('Accuracy',fontsize=16)
    plt.title('Accuracy Curves',fontsize=16)
    plt.savefig('fig2.png')

In [ ]:
history()

In [ ]:
# download model
from google.colab import files
files.download('fig1.png')
files.download('fig2.png')
files.download('pytorch_cifar10_model.pth')